In [234]:
# 토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# LangSmith 추적을 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("review_sentiment_7")

LangSmith 추적을 시작합니다.
[프로젝트명]
review_sentiment_7


In [237]:
# llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.3, model_name="gpt-4o")

In [ ]:
# 데이터 가져오기
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/review/비외식업 감정/review_negative_비외식업_세탁.csv",
    encoding="utf-8",
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   가게명     15 non-null     object 
 1   카테고리    15 non-null     object 
 2   리뷰      15 non-null     object 
 3   원본 리뷰   15 non-null     object 
 4   감성      15 non-null     object 
 5   감성 스코어  15 non-null     float64
dtypes: float64(1), object(5)
memory usage: 852.0+ bytes


In [273]:
# 서비스, 위생 데이터셋 구분
service_df = df[df["카테고리"].str.contains("서비스", na=False)]
hygiene_df = df[df["카테고리"].str.contains("위생", na=False)]

service_df.info()
hygiene_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 0 to 11
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   가게명     9 non-null      object 
 1   카테고리    9 non-null      object 
 2   리뷰      9 non-null      object 
 3   원본 리뷰   9 non-null      object 
 4   감성      9 non-null      object 
 5   감성 스코어  9 non-null      float64
dtypes: float64(1), object(5)
memory usage: 504.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 1 to 14
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   가게명     6 non-null      object 
 1   카테고리    6 non-null      object 
 2   리뷰      6 non-null      object 
 3   원본 리뷰   6 non-null      object 
 4   감성      6 non-null      object 
 5   감성 스코어  6 non-null      float64
dtypes: float64(1), object(5)
memory usage: 336.0+ bytes


In [281]:
# input 데이터 로드
input_review = service_df["리뷰"]

input_review_test = input_review
input_review_test = input_review_test.to_list()

len(input_review_test)

9

### GPT-4o 사용

In [282]:
# prompt 가져오기
from langchain_core.prompts import load_prompt

prompt = load_prompt(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/review/sentiment_service_fewshot.yaml",
    encoding="utf-8",
)
prompt

FewShotPromptTemplate(input_variables=['format_instructions', 'input_review_test'], input_types={}, partial_variables={}, examples=[{'input_review_test': '사장님의 조금 덜 친절하세요.', 'output': '{{\n  "score": 0.17,\n  "reason": "서비스에 대한 약한 아쉬움 표현으로 부정 감정이 있으나 경미함"\n}} \n'}, {'input_review_test': '직원이 불친절하게 대해줘서 기분이 나빠요', 'output': '{{\n  "score": 0.43,\n  "reason": "서비스 문제에 대한 직접적인 언급과 기분 표현이 있으며, 강한 표현은 아님"\n}} \n'}, {'input_review_test': '직원 분이 정말 불친절하게 응대하시네요. 화가 났어요', 'output': '{{\n  "score": 0.68,\n  "reason": "강한 표현 \'정말 별로\' 사용으로 높은 수준의 부정 감정"\n}}\n'}, {'input_review_test': '오히려 사장님이 화를 내시네요. 정말 다시는 가고 싶지 않습니다. 최악이에요.', 'output': '{{\n  "score": 0.85,\n  "reason": "강도 높은 혐오 표현의 반복과 극단적인 단절 의사로 인해 매우 강한 부정 감정"\n}} \n'}], example_prompt=PromptTemplate(input_variables=['input_review_test', 'output'], input_types={}, partial_variables={}, template='리뷰: {input_review_test}\n#Answer:\n{output}\n'), suffix='리뷰: {input_review_test}\n\n#Answer:\n', prefix='당신은 한국어 리뷰에 대한 감정의 강도를 분석하는 전문 감정 분석가입니

In [ ]:
# 출력 데이터 모델 정의 (score, reason)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional


class SentimentResult(BaseModel):
    score: Optional[float] = Field(
        description="0~1 사이의 감정 점수. 1에 가까울수록 강한 부정 감정."
    )
    reason: str = Field(description="해당 점수를 준 이유를 짧게 서술.")


output_parser = PydanticOutputParser(pydantic_object=SentimentResult)

In [284]:
format_instructions = output_parser.get_format_instructions()
prompt = prompt.partial(format_instructions=format_instructions)

In [285]:
# 체인 생성
chain = prompt | llm | output_parser

In [ ]:
# chain 실행 및 결과 출력
results = []
for review in input_review_test:
    result = chain.invoke(
        {
            "input_review_test": review,
            "format_instructions": output_parser.get_format_instructions(),
        }
    )
    results.append({"review": review, "score": result.score, "reason": result.reason})

In [ ]:
# 데이터프레임 형식으로 저장
df_results = pd.DataFrame(results)

df_results.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/review/scoring_data/0428/비외식업_세탁_서비스점수.csv",
    index=False,
    encoding="utf-8-sig",
)